# Exercise 2 - More comparative evaluation

## Import dependencies

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import time
import pandas as pd
import glob, os
from sklearn import preprocessing
import matplotlib.pyplot as plt
from librosa import display
import librosa
import numpy as np
from collections import deque
import progressbar
from sklearn.model_selection import cross_val_score
from sklearn import neighbors
from sklearn import naive_bayes
from sklearn import tree
from sklearn import ensemble
from sklearn import svm
import numpy as np
import scipy.stats.stats as st
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

matrikelNumber = 11776836
isEvenMatrikelNumber = matrikelNumber % 2 == 0
dataset = "music" if True else "image"
print("The", dataset, "dataset is used!")

The music dataset is used!


## Load datasets

### Small dataset: HR-dataset

In [5]:
hrDataset = pd.read_csv('../data/HRDataset_v14.csv', sep=',')
hrDataset.sample(frac=1, random_state=matrikelNumber).reset_index(drop=True)

# Clean the dataset
print(
    "I drop all continuous columns from the dataset, I can't encode these columns well. Categorization of these columns will not work, because every value is very different.")
filteredHR = hrDataset.drop("EmpID", axis=1)
filteredHR = filteredHR.drop("Employee_Name", axis=1)
filteredHR = filteredHR.drop("DateofTermination", axis=1)
filteredHR = filteredHR.drop("LastPerformanceReview_Date", axis=1)
filteredHR = filteredHR.drop("DateofHire", axis=1)
filteredHR = filteredHR.drop("EngagementSurvey", axis=1)
filteredHR = filteredHR.drop("Salary", axis=1)
filteredHR = filteredHR.drop("DOB", axis=1)

filteredHR['absences_label'] = filteredHR['Absences'].apply(lambda value: '0-14'
if value <= 4 else '0-5'
if value <= 5 else '6-10'
if value <= 10 else '11-15'
if value <= 14 else '15+')

filteredHR = filteredHR.drop("Absences", axis=1)
# filteredHR.head()

# One hot encode the dataset
encodedHR = pd.get_dummies(filteredHR, columns=filteredHR.columns, drop_first=True)

# Split the dataset
hrXAxis = encodedHR.drop('RaceDesc_White', axis=1)
hrYAxis = encodedHR['RaceDesc_White']

encodedHR.head()

I drop all continuous columns from the dataset, I can't encode these columns well. Categorization of these columns will not work, because every value is very different.


,MarriedID_1,MaritalStatusID_1,MaritalStatusID_2,MaritalStatusID_3,MaritalStatusID_4,GenderID_1,EmpStatusID_2,EmpStatusID_3,EmpStatusID_4,EmpStatusID_5,DeptID_2,DeptID_3,DeptID_4,DeptID_5,DeptID_6,PerfScoreID_2,PerfScoreID_3,PerfScoreID_4,FromDiversityJobFairID_1,Termd_1,PositionID_2,PositionID_3,PositionID_4,PositionID_5,PositionID_6,PositionID_7,PositionID_8,PositionID_9,PositionID_10,PositionID_11,PositionID_12,PositionID_13,PositionID_14,PositionID_15,PositionID_16,PositionID_17,PositionID_18,PositionID_19,PositionID_20,PositionID_21,PositionID_22,PositionID_23,PositionID_24,PositionID_25,PositionID_26,PositionID_27,PositionID_28,PositionID_29,PositionID_30,Position_Administrative Assistant,Position_Area Sales Manager,Position_BI Developer,Position_BI Director,Position_CIO,Position_Data Analyst,Position_Data Analyst,Position_Data Architect,Position_Database Administrator,Position_Director of Operations,Position_Director of Sales,Position_Enterprise Architect,Position_IT Director,Position_IT Manager - DB,Position_IT Manager - Infra,Position_IT Manager - Support,Position_IT Support,Position_Network Engineer,Position_President & CEO,Position_Principal Data Architect,Position_Production Manager,Position_Production Technician I,Position_Production Technician II,Position_Sales Manager,Position_Senior BI Developer,Position_Shared Services Manager,Position_Software Engineer,Position_Software Engineering Manager,Position_Sr. Accountant,Position_Sr. DBA,Position_Sr. Network Engineer,State_AZ,State_CA,State_CO,State_CT,State_FL,State_GA,State_ID,State_IN,State_KY,State_MA,State_ME,State_MT,State_NC,State_ND,State_NH,State_NV,State_NY,State_OH,State_OR,State_PA,State_RI,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,Zip_1040,Zip_1420,Zip_1450,Zip_1460,Zip_1545,Zip_1550,Zip_1701,Zip_1721,Zip_1730,Zip_1731,Zip_1742,Zip_1749,Zip_1752,Zip_1760,Zip_1770,Zip_1773,Zip_1775,Zip_1776,Zip_1778,Zip_1801,Zip_1803,Zip_1810,Zip_1821,Zip_1824,Zip_1844,Zip_1845,Zip_1851,Zip_1864,Zip_1876,Zip_1880,Zip_1886,Zip_1887,Zip_1890,Zip_1901,Zip_1902,Zip_1905,Zip_1915,Zip_1938,Zip_1960,Zip_2021,Zip_2026,Zip_2027,Zip_2030,Zip_2031,Zip_2043,Zip_2044,Zip_2045,Zip_2048,Zip_2050,Zip_2056,Zip_2061,Zip_2062,Zip_2066,Zip_2067,Zip_2072,Zip_2081,Zip_2090,Zip_2093,Zip_2108,Zip_2109,Zip_2110,Zip_2119,Zip_2121,Zip_2122,Zip_2124,Zip_2125,Zip_2126,Zip_2127,Zip_2128,Zip_2129,Zip_2130,Zip_2132,Zip_2133,Zip_2134,Zip_2135,Zip_2136,Zip_2138,Zip_2140,Zip_2143,Zip_2145,Zip_2148,Zip_2149,Zip_2152,Zip_2154,Zip_2155,Zip_2169,Zip_2170,Zip_2171,Zip_2176,Zip_2180,Zip_2184,Zip_2189,Zip_2191,Zip_2199,Zip_2210,Zip_2301,Zip_2302,Zip_2324,Zip_2330,Zip_2343,Zip_2346,Zip_2351,Zip_2359,Zip_2360,Zip_2421,Zip_2445,Zip_2446,Zip_2451,Zip_2452,Zip_2453,Zip_2458,Zip_2459,Zip_2468,Zip_2472,Zip_2474,Zip_2478,Zip_2481,Zip_2492,Zip_2493,Zip_2703,Zip_2718,Zip_2747,Zip_2763,Zip_2908,Zip_3062,Zip_4063,Zip_5473,Zip_5664,Zip_6033,Zip_6040,Zip_6050,Zip_6070,Zip_6278,Zip_10171,Zip_19444,Zip_21851,Zip_27229,Zip_30428,Zip_33174,Zip_36006,Zip_37129,Zip_40220,Zip_43050,Zip_46204,Zip_58782,Zip_59102,Zip_78207,Zip_78230,Zip_78789,Zip_80820,Zip_83706,Zip_84111,Zip_85006,Zip_89139,Zip_90007,Zip_97756,Zip_98052,Sex_M,MaritalDesc_Married,MaritalDesc_Separated,MaritalDesc_Single,MaritalDesc_Widowed,CitizenDesc_Non-Citizen,CitizenDesc_US Citizen,HispanicLatino_Yes,HispanicLatino_no,HispanicLatino_yes,RaceDesc_Asian,RaceDesc_Black or African American,RaceDesc_Hispanic,RaceDesc_Two or more races,RaceDesc_White,TermReason_Fatal attraction,TermReason_Learned that he is a gangster,TermReason_N/A-StillEmployed,TermReason_attendance,TermReason_career change,TermReason_gross misconduct,TermReason_hours,TermReason_maternity leave - did not return,TermReason_medical issues,TermReason_military,TermReason_more money,"TermReason_no-call, no-show",TermReason_performance,TermReason_relocation out of area,TermReason_retiring,TermReason_return to school,TermReason_unhappy,EmploymentStatus_Terminated for Cause,EmploymentStatus_Volunta

### Large dataset: Census income

In [6]:
censusIncome = pd.read_csv('../data/census-income.data', sep=',')
censusIncome.sample(frac=1, random_state=matrikelNumber).reset_index(drop=True)

censusIncome['age_label'] = censusIncome['age'].apply(lambda value: '0-14'
if value <= 14 else '15-24'
if value <= 24 else '25-54'
if value <= 54 else '55-64'
if value <= 64 else '65 years and over')

censusIncome['wage_label'] = censusIncome['wage per hour'].apply(lambda value: '0-3000'
if value <= 3000 else '3001-6000'
if value <= 6000 else '6001-7000'
if value <= 7000 else '7001-9000+')

filteredCensus = censusIncome[
    (censusIncome['class of worker'] != ' Not in universe') &
    (censusIncome['education'] != ' Children') &
    (censusIncome['wage per hour'] > 0) &
    (censusIncome['weeks worked in year'] > 0)
    ]

filteredCensus = filteredCensus.drop("age", axis=1)
filteredCensus = filteredCensus.drop("wage per hour", axis=1)
filteredCensus = filteredCensus.drop("year", axis=1)
filteredCensus = filteredCensus.drop("ignore", axis=1)
filteredCensus = filteredCensus.drop("instance weight", axis=1)

# filteredCensus.head()

# One hot encode the dataset
encodedIncome = pd.get_dummies(filteredCensus, columns=filteredCensus.columns, drop_first=True)

# Split the dataset
censusIncomeXAxis = encodedIncome.drop('race_ White', axis=1)
censusIncomeYAxis = encodedIncome['race_ White']

encodedIncome.head()

,class of worker_ Local government,class of worker_ Private,class of worker_ State government,detailed industry recode_2,detailed industry recode_3,detailed industry recode_4,detailed industry recode_5,detailed industry recode_6,detailed industry recode_7,detailed industry recode_8,detailed industry recode_9,detailed industry recode_11,detailed industry recode_12,detailed industry recode_13,detailed industry recode_14,detailed industry recode_15,detailed industry recode_16,detailed industry recode_17,detailed industry recode_18,detailed industry recode_19,detailed industry recode_20,detailed industry recode_21,detailed industry recode_22,detailed industry recode_23,detailed industry recode_24,detailed industry recode_25,detailed industry recode_26,detailed industry recode_27,detailed industry recode_28,detailed industry recode_29,detailed industry recode_30,detailed industry recode_31,detailed industry recode_32,detailed industry recode_33,detailed industry recode_34,detailed industry recode_35,detailed industry recode_36,detailed industry recode_37,detailed industry recode_38,detailed industry recode_39,detailed industry recode_40,detailed industry recode_41,detailed industry recode_42,detailed industry recode_43,detailed industry recode_44,detailed industry recode_45,detailed industry recode_46,detailed industry recode_47,detailed industry recode_48,detailed industry recode_49,detailed industry recode_50,detailed occupation recode_2,detailed occupation recode_3,detailed occupation recode_4,detailed occupation recode_5,detailed occupation recode_6,detailed occupation recode_7,detailed occupation recode_8,detailed occupation recode_9,detailed occupation recode_10,detailed occupation recode_11,detailed occupation recode_12,detailed occupation recode_13,detailed occupation recode_14,detailed occupation recode_15,detailed occupation recode_16,detailed occupation recode_17,detailed occupation recode_18,detailed occupation recode_19,detailed occupation recode_20,detailed occupation recode_21,detailed occupation recode_22,detailed occupation recode_23,detailed occupation recode_24,detailed occupation recode_25,detailed occupation recode_26,detailed occupation recode_27,detailed occupation recode_28,detailed occupation recode_29,detailed occupation recode_30,detailed occupation recode_31,detailed occupation recode_32,detailed occupation recode_33,detailed occupation recode_34,detailed occupation recode_35,detailed occupation recode_36,detailed occupation recode_37,detailed occupation recode_38,detailed occupation recode_39,detailed occupation recode_40,detailed occupation recode_41,detailed occupation recode_42,detailed occupation recode_43,detailed occupation recode_44,detailed occupation recode_45,education_ 11th grade,education_ 12th grade no diploma,education_ 1st 2nd 3rd or 4th grade,education_ 5th or 6th grade,education_ 7th and 8th grade,education_ 9th grade,education_ Associates degree-academic program,education_ Associates degree-occup /vocational,education_ Bachelors degree(BA AB BS),education_ Doctorate degree(PhD EdD),education_ High school graduate,education_ Less than 1st grade,education_ Masters degree(MA MS MEng MEd MSW MBA),education_ Prof school degree (MD DDS DVM LLB JD),education_ Some college but no degree,enroll in edu inst last wk_ High school,enroll in edu inst last wk_ Not in universe,marital stat_ Married-A F spouse present,marital stat_ Married-civilian spouse present,marital stat_ Married-spouse absent,marital stat_ Never married,marital stat_ Separated,marital stat_ Widowed,major industry code_ Business and repair services,major industry code_ Communications,major industry code_ Construction,major industry code_ Education,major industry code_ Entertainment,major industry code_ Finance insurance and real estate,major industry code_ Forestry and fisheries,major industry code_ Hospital services,major industry code_ Manufacturing-durable goods,major industry code_ Manufacturing-nondurable goods,major industry

### Music dataset

In [7]:
# We need to construct our data set; unfortunately, we don't simply have a "loadGTZanDataSet()" function in SK-learn...
# So we need to
## Download our data set & extract it (one-time effort)
## Run an audio feature extraction
## Create the create the ground truth (label assignment, target, ...)


# path to our audio folder
# For the first run, download the images from http://kronos.ifs.tuwien.ac.at/GTZANmp3_22khz.zip, and unzip them to your folder
imagePath = "../data/GTZANmp3_22khz/"

# Find all songs in that folder; there are like 1.000 different ways to do this in Python, we chose this one :-)
os.chdir(imagePath)
fileNames = glob.glob("*/*.mp3")
numberOfFiles = len(fileNames)
targetLabels = []

print("Found " + str(numberOfFiles) + " files\n")

# The first step - create the ground truth (label assignment, target, ...)
# For that, iterate over the files, and obtain the class label for each file
# Basically, the class name is in the full path name, so we simply use that
for fileName in fileNames:
    pathSepIndex = fileName.index("/")
    targetLabels.append(fileName[:pathSepIndex])

# sk-learn can only handle labels in numeric format - we have them as strings though...
# Thus we use the LabelEncoder, which does a mapping to Integer numbers
le = preprocessing.LabelEncoder()
le.fit(targetLabels)  # this basically finds all unique class names, and assigns them to the numbers
print("Found the following classes: " + str(list(le.classes_)))

# now we transform our labels to integers
target = le.transform(targetLabels)
print("Transformed labels (first elements: " + str(target[0:150]))

# If we want to find again the label for an integer value, we can do something like this:
# print list(le.inverse_transform([0, 18, 1]))

print("... done label encoding")

Found 1000 files

Found the following classes: ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
Transformed labels (first elements: [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1]
... done label encoding


## Define Classifiers

Define functions for running the classifiers on the data. I define a function for running kNN, perceptron, decision tree, svm and random forests.

### k-NN (k-nearest-neighbours)

In [8]:
def kNearestNeighbours(datasetName, kNeighbours, XTrain, XTest, yTrain, yTest):
    results = []

    for k in kNeighbours:
        print('kNN with', k, 'neighbours')
        knnClassifier = KNeighborsClassifier(n_neighbors=k, algorithm='kd_tree')

        # Train the classifier
        startTime = time.time()
        knnClassifier.fit(XTrain, yTrain.ravel())
        endTime = time.time()
        trainTime = endTime - startTime

        # Predict
        startTime = time.time()
        predicted = knnClassifier.predict(XTest)
        endTime = time.time()
        predictionTime = endTime - startTime

        # Effectiveness measurement
        accuracyScore = accuracy_score(yTest, predicted)
        f1Score = f1_score(yTest, predicted, average='weighted')

        print('Training time:', trainTime, 's')
        print('Testing time:', predictionTime, 's')
        print()

        print('Accuracy:', accuracyScore * 100, '%')
        print('F1 score:', f1Score * 100, '%')
        print()

        result = {
            'datasetName': datasetName,
            'neighbours': k,
            'accuracyScore': accuracyScore,
            'f1Score': f1Score,
            'trainTime': trainTime,
            'predictionTime': predictionTime
        }

        results.append(result)

    return results

### Perceptron

In [9]:
def perceptron(datasetName, alphaValues, XTrain, XTest, yTrain, yTest):
    results = []

    for alpha in alphaValues:
        print('Perceptron with alpha', alpha)
        perceptronClassifier = Perceptron(alpha=alpha, random_state=matrikelNumber)

        # Train the classifier
        startTime = time.time()
        perceptronClassifier.fit(XTrain, yTrain.ravel())
        endTime = time.time()
        trainTime = endTime - startTime

        # Predict
        startTime = time.time()
        predicted = perceptronClassifier.predict(XTest)
        endTime = time.time()
        predictionTime = endTime - startTime

        # Effectiveness measurement
        accuracyScore = accuracy_score(yTest, predicted)
        f1Score = f1_score(yTest, predicted, average='weighted')

        print('Training time:', trainTime, 's')
        print('Testing time:', predictionTime, 's')
        print()

        print('Accuracy:', accuracyScore * 100, '%')
        print('F1 score:', f1Score * 100, '%')
        print()

        result = {
            'datasetName': datasetName,
            'alpha': alpha,
            'accuracyScore': accuracyScore,
            'f1Score': f1Score,
            'trainTime': trainTime,
            'predictionTime': predictionTime
        }
        results.append(result)

    return results

### Decision tree

In [10]:
def decisionTree(datasetName, maxFeatureValues, XTrain, XTest, yTrain, yTest):
    results = []

    for maxFeatures in maxFeatureValues:
        print('Decision Tree with max features', maxFeatures)

        decisionTreeClassifier = DecisionTreeClassifier(max_features=maxFeatures, random_state=matrikelNumber)

        # Train the classifier
        startTime = time.time()
        decisionTreeClassifier.fit(XTrain, yTrain.ravel())
        endTime = time.time()
        trainTime = endTime - startTime

        # Predict
        startTime = time.time()
        predicted = decisionTreeClassifier.predict(XTest)
        endTime = time.time()
        predictionTime = endTime - startTime

        # Effectiveness measurement
        accuracyScore = accuracy_score(yTest, predicted)
        f1Score = f1_score(yTest, predicted, average='weighted')

        result = {
            'datasetName': datasetName,
            'maxFeatures': maxFeatures,
            'accuracyScore': accuracyScore,
            'f1Score': f1Score,
            'trainTime': trainTime,
            'predictionTime': predictionTime
        }
        results.append(result)

    return results

### SVM (Support vector machine)

In [11]:
def supportVectorMachine(datasetName, XTrain, XTest, yTrain, yTest):
    print('Support Vector Machine')

    results = []

    svmClassifier = make_pipeline(StandardScaler(), SVC(random_state=matrikelNumber))

    # Train the classifier
    startTime = time.time()
    svmClassifier.fit(XTrain, yTrain.ravel())
    endTime = time.time()
    trainTime = endTime - startTime

    # Predict
    startTime = time.time()
    predicted = svmClassifier.predict(XTest)
    endTime = time.time()
    predictionTime = endTime - startTime

    # Effectiveness measurement
    accuracyScore = accuracy_score(yTest, predicted)
    f1Score = f1_score(yTest, predicted, average='weighted')

    result = {
        'datasetName': datasetName,
        'accuracyScore': accuracyScore,
        'f1Score': f1Score,
        'trainTime': trainTime,
        'predictionTime': predictionTime
    }
    results.append(result)

    return results

### Random forests

In [12]:
def randomForest(datasetName, numberOfTrees, maxFeatureValues, XTrain, XTest, yTrain, yTest):
    results = []

    for numberOfTreeElements in numberOfTrees:
        for maxFeatureValue in maxFeatureValues:
            print('Random forest with', numberOfTreeElements, 'trees and ', maxFeatureValue, 'max features')

            randomForestClassifier = RandomForestClassifier(
                n_estimators=numberOfTreeElements,
                max_features=maxFeatureValue,
                random_state=matrikelNumber
            )

            # Train the classifier
            startTime = time.time()
            randomForestClassifier.fit(XTrain, yTrain.ravel())
            endTime = time.time()
            trainTime = endTime - startTime

            # Predict
            startTime = time.time()
            predicted = randomForestClassifier.predict(XTest)
            endTime = time.time()
            predictionTime = endTime - startTime

            # Effectiveness measurement
            accuracyScore = accuracy_score(yTest, predicted)
            f1Score = f1_score(yTest, predicted, average='weighted')

            result = {
                'datasetName': datasetName,
                'numberOfTreeElements': numberOfTreeElements,
                'maxFeatureValue': maxFeatureValue,
                'accuracyScore': accuracyScore,
                'f1Score': f1Score,
                'trainTime': trainTime,
                'predictionTime': predictionTime
            }

            results.append(result)
    return results

## Apply Classifiers

Now we apply our classifiers to our dataset.

In [13]:
kNNResults = kNearestNeighbours()
perceptronResults = perceptron()
decisionTreeResults = decisionTree()
supportVectorMachineResults = supportVectorMachine()
randomForestResults = randomForest()

TypeError: kNearestNeighbours() missing 6 required positional arguments: 'datasetName', 'kNeighbours', 'XTrain', 'XTest', 'yTrain', and 'yTest'